TEAM MEMBERS:

1) MOHAMMAD SYAMEER IMRAN BIN MOHD ASHROF (2015807)

2) MUHAMMAD HARITH DANIAL BIN HARUN (1913029)





**INFO:**

**LEVEL:** The distance between the sensor and the water surface. The lower the number, the closer is the gap.

**INTENSITY:** The intensity of the rain during the sensor reading time. There are ***none(value 0), light(value 1), moderate(value 2), and heavy(value 3).***

**STATUS:** The status of flooding, which are ***No(value 0) and Yes(value 1).***


In [4]:
import sys
import pandas as pd
import numpy as np
import keras, matplotlib, sklearn
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
df = pd.read_csv("Data/Water Level Training Data.csv")
df.head()

,created_at,entry_id,level,intensity,status
0,2023-05-06 08:47:10 UTC,2,25.477,none,no
1,2023-05-09 06:49:51 UTC,3,25.477,none,no
2,2023-05-09 06:53:01 UTC,4,25.477,none,no
3,2023-05-09 06:56:41 UTC,5,25.477,none,no
4,2023-05-09 07:33:52 UTC,6,25.407,light,no


In [6]:
interested_columns = ['level', 'intensity', 'status']
df = df.loc[:, interested_columns]
df.head()

,level,intensity,status
0,25.477,none,no
1,25.477,none,no
2,25.477,none,no
3,25.477,none,no
4,25.407,light,no


In [7]:
# For intensity
df.replace('none','0', inplace=True)
df.replace('light','1', inplace=True)
df.replace('moderate','2', inplace=True)
df.replace('heavy','3', inplace=True)

# For status
df.replace('no', '0', inplace=True)
df.replace('yes','1', inplace=True)

df.tail()

,level,intensity,status
95,21.097,0,0
96,21.097,0,0
97,20.825,0,0
98,20.825,0,0
99,20.859,0,0


In [8]:
# Sequential Regression

input = df.loc[:, ['level', 'intensity']]
output = df.loc[:, ['status']]
X = np.asarray(input).astype('float32')
y = np.asarray(output).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(75, 2) (25, 2) (75, 1) (25, 1)


In [9]:
model = Sequential()
model.add(Dense(4, kernel_initializer = 'uniform', activation = 'relu', input_shape=[X.shape[1]]))
model.add(Dense(8, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'uniform'))
model.compile(optimizer = 'adam', loss='msle')
model.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8/8 [==============================] - 1s 3ms/step - loss: 0.2083
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 0.2014
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1933
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: 0.1835
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1727
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 0.1612
Epoch 7/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1528
Epoch 8/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1459
Epoch 9/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1435
Epoch 10/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1412
Epoch 11/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1400
Epoch 12/100
8/8 [==============================] - 0s 2ms/step - loss: 0.1389
Epoch 13/100
8/8 [==============================] - 0s 2ms/st

In [10]:
# Loss evaluation
test_loss = model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 224ms/step - loss: 0.0282


In [11]:
# Trying a new prediction[water level, intensity]
prediction =  [10.2, 3.0]
pred = model.predict([prediction])
print('Predicted: %.3f' % pred)

# Test with water level = 10cm and intensity of none which is 0.0, got 0% chance of flood.
# Test with water level = 10cm and intensity of light which is 1.0, got 20% chance of flood.
# Test with water level = 10cm and intensity of moderate which is 2.0, got 63% chance of flood.
# Test with water level = 10cm and intensity of heavy which is 3.0, got 100% chance of flood.

1/1 [==============================] - 0s 155ms/step
Predicted: 1.124


**TRYING TO UPDATE REAL-TIME THROUGH THE USAGE OF THINGSPEAK API (RESTFUL)**

In [12]:
import requests
url = ('https://api.thingspeak.com/channels/2135773/feeds.json?api_key=5ISF03WS3CO2HYSD&results=2')
print(url)

# Print status code (and associated text)
response = requests.get(url)
print(f"Request returned {response.status_code} : '{response.reason}'")

https://api.thingspeak.com/channels/2135773/feeds.json?api_key=5ISF03WS3CO2HYSD&results=2
Request returned 200 : 'OK'


In [13]:
msg=requests.get(url)
msg=msg.json()['feeds'][-1]['field2']
print("Last updated water level: \n\n"+str(msg))

Last updated water level: 

17.66300


In [14]:
# Trying a new prediction through Thingspeak API[water level, intensity]
level = float(msg)
prediction =  [level, 3.0]
pred = model.predict([prediction])
pred_int = int(pred*100)
print('Flood Probability:', pred_int,'%')

1/1 [==============================] - 0s 87ms/step
Flood Probability: 73 %


In [15]:
# Saving the model
import joblib
filename = 'model.pkl'
joblib.dump(model, filename)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-06-10 21:36:00         1894
metadata.json                                  2023-06-10 21:36:00           64
variables.h5                                   2023-06-10 21:36:00        20512


['model.pkl']